# fliter bad matches

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download_files = [
          {'id':'1n1cIwfqqUIYipKI6Gn8tVwS9XRNNYW8q','file_name':'chat.csv'},
          
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
refine_toxicity_words = []
for line in open("refined_toxicity_lexicon.txt"):
  refine_toxicity_words.append(line[:-1])
len(refine_toxicity_words)

1504

In [ ]:
refine_toxicity_words[-1] = 'noobi'

In [ ]:
refine_toxicity_words = list(set(refine_toxicity_words))
len(refine_toxicity_words)

1503

In [ ]:
import re
def remove_punctuation(x):
  x = re.sub(r"[^a-zA-Z\s]",'',x)
  return x

df_whole_chat = pd.read_csv('chat.csv')
bad_match_id1 = []
for index, row in df_whole_chat.iterrows():
  msg = str(row['key'])
  if row['match_id'] in bad_match_id1:
    continue
  msg = remove_punctuation(msg)
  msg = msg.lower()
  for word in msg.split():
    if word in refine_toxicity_words:
      bad_match_id1.append(row['match_id'])
      break
len(bad_match_id1)

30204

In [ ]:
matches_data_after_fliter = []
for index, row in df_whole_chat.iterrows():
  if row['match_id'] in bad_match_id1:
    matches_data_after_fliter.append([row['match_id'],row['key'],row['slot'],row['time'],row['unit']])
from pandas.core.frame import DataFrame
bad_match_new_lexicon = DataFrame(matches_data_after_fliter,columns=['match_id','key','slot','time','unit'])
bad_match_new_lexicon.head()

,match_id,key,slot,time,unit
0,0,force it,6,-8,6k Slayer
1,0,space created,1,5,Monkey
2,0,hah,1,6,Monkey
3,0,ez 500,6,9,6k Slayer
4,0,mvp ulti,4,934,Kira


In [ ]:
bad_match_new_lexicon.to_csv('refined_lexicon_flitered_match.csv')
print('using new lexicon data size:',bad_match_new_lexicon.shape[0])

using new lexicon data size: 1139729


# generate utterance level data and word level data

In [ ]:
import re
df_bad_match = pd.read_csv('refined_lexicon_flitered_match.csv')
utterance_list = df_bad_match['key'].tolist()
regex1 = r'[а-яА-Я]{1,}'
regex1 = re.compile(regex1)
regex2 = re.compile(u"[\u4e00-\u9fa5]+")
new_result = []
for index, row in df_bad_match.iterrows():
  utter = row['key']
  utter = str(utter)
  if not re.search(regex1,utter) and '????' not in utter:
    # print(utter)
    new_result.append([row['match_id'],row['key'],row['slot'],row['time'],row['unit']])
from pandas.core.frame import DataFrame
df_bad_match_remove_useless = DataFrame(new_result,columns=['match_id','key','slot','time','unit'])

In [ ]:
df_bad_match_remove_useless.shape[0]

1097463

In [ ]:
df_bad_match = df_bad_match_remove_useless
conversation_ids = []
conversation_id = -1
current_match_id = -1
for index, row in df_bad_match.iterrows():
  match_id = row['match_id']
  if match_id == current_match_id:
    lag = row['time'] - last_time
    if lag >60:
      conversation_id +=1
    conversation_ids.append(conversation_id)
    current_match_id = match_id
    last_time = row['time']
  else:
    conversation_id +=1
    conversation_ids.append(conversation_id)
    last_time = row['time']
    current_match_id = match_id
df_bad_match_remove_useless['conversation_id'] = conversation_ids
df_bad_match_remove_useless.head()

,match_id,key,slot,time,unit,conversation_id
0,0,force it,6,-8,6k Slayer,0
1,0,space created,1,5,Monkey,0
2,0,hah,1,6,Monkey,0
3,0,ez 500,6,9,6k Slayer,0
4,0,mvp ulti,4,934,Kira,1


In [ ]:
# These are just common English contractions. There are many edge cases. i.e. University's working on it.
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [ ]:
## constract restore
def contraction_restore(x):
  x = str(x)
  new_str = ''
  for word in x.split():
    if word in contraction_dict:
      new_str += contraction_dict[word] + ' '
    else:
      new_str += word + ' '
  return new_str
df_bad_match_remove_useless['key_after_contraction'] = df_bad_match_remove_useless['key'].map(contraction_restore)
df_bad_match_remove_useless.head()

,match_id,key,slot,time,unit,conversation_id,key_after_contraction
0,0,force it,6,-8,6k Slayer,0,force it
1,0,space created,1,5,Monkey,0,space created
2,0,hah,1,6,Monkey,0,hah
3,0,ez 500,6,9,6k Slayer,0,ez 500
4,0,mvp ulti,4,934,Kira,1,mvp ulti


In [ ]:
### separate
import re
def separate_special_char_for_each_word(x):
  regex = re.compile(r"[a-zA-Z][^:;=a-zA-Z\s\t]")
  if re.search(regex,x):
    result = re.search(regex,x)
    a = list(x)
    a.insert(result.start()+1, ' ')
    x = ''.join(a)
  regex = re.compile(r"[^:;=a-zA-Z\s\t<][a-zA-Z]")
  if re.search(regex,x):
    result = re.search(regex,x)
    a = list(x)
    a.insert(result.start()+1, ' ')
    x = ''.join(a)
  return x

def separate_special_char_for_each_word_revise(x):
  regex1 = re.compile(r"[a-zA-Z][^a-zA-Z\s\t0-9]")
  regex2 = re.compile(r"[^a-zA-Z\s\t0-9:=][a-zA-Z]")
  while re.search(regex1,x) or re.search(regex2,x):
    if re.search(regex1,x):
      result = re.search(regex1,x)
      a = list(x)
      a.insert(result.start()+1, ' ')
      x = ''.join(a)
    if re.search(regex2,x):
      result = re.search(regex2,x)
      a = list(x)
      a.insert(result.start()+1, ' ')
      x = ''.join(a)
  return x

def separate_special_char(x):
  new_utterance = ''
  for word in str(x).split():
    word = separate_special_char_for_each_word(word)
    new_utterance+=word + ' '
  return new_utterance

In [ ]:
df_bad_match_remove_useless['key_after_separation'] = df_bad_match_remove_useless['key_after_contraction'].map(separate_special_char_for_each_word_revise)
df_bad_match_remove_useless.head()

,match_id,key,slot,time,unit,conversation_id,key_after_contraction,key_after_separation
0,0,force it,6,-8,6k Slayer,0,force it,force it
1,0,space created,1,5,Monkey,0,space created,space created
2,0,hah,1,6,Monkey,0,hah,hah
3,0,ez 500,6,9,6k Slayer,0,ez 500,ez 500
4,0,mvp ulti,4,934,Kira,1,mvp ulti,mvp ulti


In [ ]:
## remove special characters
regex_init = "[^:;=][^a-zA-Z0-9\s:=<;]"
regex_wenhao = "[?]"
regex_chinese = re.compile(u"[\u4e00-\u9fa5]+")
regex_rec = "[]"
regex_first = "^[^a-zA-Z0-9\s:=<;]"
def remove_special_chars(y):
  utter = str(y)
  new_result = ''
  for word in utter.split():
    # print(word)
    new_utter = re.sub(regex_init,'',word)
    new_utter = re.sub(regex_wenhao,'',new_utter)
    new_utter = re.sub(regex_chinese,'',new_utter)
    new_utter = re.sub(regex_rec,'',new_utter)
    new_utter = re.sub(regex_first,'',new_utter)
    new_utter = new_utter.strip()
    if new_utter != '':
      new_result += new_utter+' '
  return new_result
df_bad_match_remove_useless['key_after_removal'] = df_bad_match_remove_useless['key_after_separation'].map(remove_special_chars)
df_bad_match_remove_useless.head()

,match_id,key,slot,time,unit,conversation_id,key_after_contraction,key_after_separation,key_after_removal
0,0,force it,6,-8,6k Slayer,0,force it,force it,force it
1,0,space created,1,5,Monkey,0,space created,space created,space created
2,0,hah,1,6,Monkey,0,hah,hah,hah
3,0,ez 500,6,9,6k Slayer,0,ez 500,ez 500,ez 500
4,0,mvp ulti,4,934,Kira,1,mvp ulti,mvp ulti,mvp ulti


In [ ]:
combined_utterance = []
last_match_id = -1
last_conversation_id = -1
for index, row in df_bad_match_remove_useless.iterrows():
  match_id = row['match_id']
  if match_id == last_match_id and last_conversation_id == row['conversation_id'] and last_user_name == row['unit'] and last_slot == row['slot'] and row['time'] - last_time <=60:
    combined_utterance[-1][2] = combined_utterance[-1][2] + ' [SEPA] '+str(row['key_after_removal'])
  else:
    combined_utterance.append([match_id,row['conversation_id'],str(row['key_after_removal']),row['time'],row['slot'],row['unit']])
    last_match_id = match_id
    last_conversation_id = row['conversation_id']
    last_user_name = row['unit']
    last_slot = row['slot']
    last_time = row['time']

In [ ]:
from pandas.core.frame import DataFrame
utterance2=DataFrame(combined_utterance,columns=['match_id','conversation_id','utterance','time','slot','unit'])
word_level_utterance = utterance2['utterance'].tolist()
utterance=DataFrame(combined_utterance,columns=['match_id','conversation_id','utterance','time','slot','unit'])
utterance['word_level_utterance'] = word_level_utterance
utterance.shape[0]

783866

In [ ]:
utterance.head()

,match_id,conversation_id,utterance,time,slot,unit,word_level_utterance
0,0,0,force it,-8,6,6k Slayer,force it
1,0,0,space created [SEPA] hah,5,1,Monkey,space created [SEPA] hah
2,0,0,ez 500,9,6,6k Slayer,ez 500
3,0,1,mvp ulti,934,4,Kira,mvp ulti
4,0,2,bye,1486,6,6k Slayer,bye


# auto labelling

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download_files = [
          {'id':'1VALiNA_ruYl0jt66MBG7Eb_o3sWQIBqj','file_name':'50K_Output.csv'},
          {'id':'1-HsJ2w944FNCZPPiqYjV7Ip-2fmdNqz5','file_name':'word_level_data_combined_annotation.csv'},
          {'id':'1vHhnY7vvA5FZemgka1-mDTE-GadI0-bq','file_name':'50K_Output_wrong_label_removed.csv'},
          {'id':'1o3CQc34m1sFSZWVCwmIlVRJoE8XItdBW','file_name':'wrong_label_in_the_dataset.csv'},
          {'id':'1CQTKmXpo1l9TFv4-juuyU7NN1bTsnr_b','file_name':'50K_Output2.csv'},
          {'id':'1EJy2AXC7ELU-6TJd4SDbi1kNGM-4b4U-','file_name':'50K_Output_from_xinghong.csv'},
          {'id':'17_RsK0dBShqjGV-NAH4W3Nt7EbdFdep6','file_name':'50K_Output_from_xinghong2.csv'},
          {'id':'1y_AlSVQhqjikF3otPvI5RwrYGv_6vo0H','file_name':'45k_cleaned.csv'},
          {'id':'1Jxi1j9aTXA7lg-tFy2tz_6uM40D7xUNk','file_name':'cleand_lexicon.csv'},
          {'id':'1SbsUfo-p5WdlvhZsCTvbUVDVyvTnjuvo','file_name':'cleand_and_refined_lexicon.csv'},
          {'id':'1TJVC6VGiAqrMzNbh_-a70FWlR6zzLuvP','file_name':'45k_add_slot_label.csv'},
          ]
for entity in download_files:
  id1 = entity['id']
  downloaded = drive.CreateFile({'id':id1 }) 
  downloaded.GetContentFile(entity['file_name']) 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# colnames = ['1','2','3','utterance','4','5','6','Label','combined_annotation']
lexicons = pd.read_csv("cleand_and_refined_lexicon.csv")
print(lexicons.head())
C_lexicon = lexicons['C'].tolist()
D_lexicon = lexicons['D'].tolist()
P_lexicon = lexicons['P'].tolist()
S_lexicon = lexicons['S'].tolist()
T_lexicon = lexicons['T'].tolist()
def fliter_empty(ll):
  new_ll = []
  for l in ll:
    if str(l) != 'nan':
      new_ll.append(str(l).lower())
  return new_ll
C_lexicon = fliter_empty(C_lexicon)
D_lexicon = fliter_empty(D_lexicon)
P_lexicon = fliter_empty(P_lexicon)
S_lexicon = fliter_empty(S_lexicon)
T_lexicon = fliter_empty(T_lexicon)
print('C lexicon size:',len(C_lexicon))
print('D lexicon size:',len(D_lexicon))
print('P lexicon size:',len(P_lexicon))
print('S lexicon size:',len(S_lexicon))
print('T lexicon size:',len(T_lexicon))

   Unnamed: 0       C       D        P          S      T
0           0      aa   25mmr      all        2ez   4r5e
1           1     aaa   abyss     alll        afk   5h1t
2           2    aaaa      ac  allllll      aggro   5hit
3           3   aaaaa   aeges  another  anchients    a55
4           4  aaaaaa  aeghis      any    ancient  a_s_s
C lexicon size: 355
D lexicon size: 386
P lexicon size: 172
S lexicon size: 284
T lexicon size: 1627


In [ ]:
C_lexicon_set = set(C_lexicon)
D_lexicon_set = set(D_lexicon)
P_lexicon_set = set(P_lexicon)
S_lexicon_set = set(S_lexicon)
T_lexicon_set = set(T_lexicon)
print('C lexicon size:',len(C_lexicon_set))
print('D lexicon size:',len(D_lexicon_set))
print('P lexicon size:',len(P_lexicon_set))
print('S lexicon size:',len(S_lexicon_set))
print('T lexicon size:',len(T_lexicon_set))

C lexicon size: 355
D lexicon size: 386
P lexicon size: 172
S lexicon size: 284
T lexicon size: 1627


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# colnames = ['1','2','3','utterance','4','5','6','Label','combined_annotation']
data = pd.read_csv("45k_cleaned.csv")

In [ ]:
import re
def get_word_level_text(x):
  x = str(x)
  if x == 'nan':
    return ''
  split_array = re.split("\(SEPA\),|\([ODPSTC]\),", x)
  final_sentence = []
  for word in split_array:
    if word == '':
      continue
    final_sentence.append(word.strip())
  return ' '.join(final_sentence)
data['word_level_data'] = data['combined_annotation'].map(get_word_level_text)

In [ ]:
def annotate(x):
  x = str(x).lower()
  if x == '':
    return ''
  annotations = ''
  for value in x.split():
    add_on = 'O '
    if value == '.':
      add_on='O '
    if value == '[sepa]':
      add_on='SEPA '
    if value in T_lexicon_set:
      add_on='T '
    if value in S_lexicon_set:
      add_on='S '
    if value in D_lexicon_set:
      add_on='D '
    if value in P_lexicon_set:
      add_on='P '
    if value in C_lexicon_set:
      add_on='C '
    annotations+=add_on
  if len(annotations.split()) != len(x.split()):
    print(x)
  return annotations
data['slot_label'] = data['word_level_data'].map(annotate)

In [ ]:
def combine_annotation(utter,anno):
  if utter == '':
    return ''
  result = ''
  utter_list = utter.split()
  anno_list = anno.split()
  assert len(utter_list) == len(anno_list)
  for i in range(len(utter_list)):
    result += utter_list[i] + ' '+ '('+anno_list[i]+'),'+' '
  return result
data['combined_annotation_version2'] = data.apply(lambda row: combine_annotation(row['word_level_data'], row['slot_label']), axis=1)
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
data.to_csv('45k_after_slot_annotation.csv')

# construct input for baseline models

In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv("45k_add_slot_label.csv")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data,  test_size=0.4, random_state=42)
X_val, X_test= train_test_split(X_test,  test_size=0.5, random_state=42)

In [ ]:
import re

train_seqin=[]

train_label=[]
train_seqout = []

train_seqin = X_train['word_level_data'].tolist()
train_label= X_train['final_label'].tolist()
train_seqout = X_train['slot_label'].tolist()
print(len(train_seqout))
print(len(train_seqin))
print(len(train_label))

pd.DataFrame(train_seqin).to_csv("./seq.in", index=False, header=None)

pd.DataFrame(train_seqout).to_csv("./seq.out", index=False, header=None)
pd.DataFrame(train_label).to_csv("./label", index=False, header=None)

In [ ]:
import re

train_seqin=[]

train_label=[]
train_seqout = []

train_seqin = X_val['word_level_data'].tolist()
train_label= X_val['final_label'].tolist()
train_seqout = X_val['slot_label'].tolist()
print(len(train_seqout))
print(len(train_seqin))
print(len(train_label))

pd.DataFrame(train_seqin).to_csv("./seq.in", index=False, header=None)

pd.DataFrame(train_seqout).to_csv("./seq.out", index=False, header=None)
pd.DataFrame(train_label).to_csv("./label", index=False, header=None)

In [ ]:
import re

train_seqin=[]

train_label=[]
train_seqout = []

train_seqin = X_test['word_level_data'].tolist()
train_label= X_test['final_label'].tolist()
train_seqout = X_test['slot_label'].tolist()
print(len(train_seqout))
print(len(train_seqin))
print(len(train_label))

pd.DataFrame(train_seqin).to_csv("./seq.in", index=False, header=None)

pd.DataFrame(train_seqout).to_csv("./seq.out", index=False, header=None)
pd.DataFrame(train_label).to_csv("./label", index=False, header=None)

## Bi-Model-Intent-And-Slot

In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv("45k_add_slot_label.csv")

In [ ]:
import re
p1 = re.compile(r'[(](.*?)[)]', re.S)
new_lines = []
for index, row in data.iterrows(): 
  words = str(row['word_level_data']).split()
  temp = str(row['slot_label']).split()
  label = row['final_label']
  assert len(words) == len(temp)
  new_line = []
  for word, slot_label in zip(words, temp):
    new_line.append(str(word).strip()+'<::>'+str(slot_label).strip())
  new_line_str = ' '.join(new_line)
  new_line_str += ' <=> '+str(label)
  new_lines.append(new_line_str)
data['new_utterance'] = new_lines

In [ ]:
from sklearn.utils import shuffle

import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data,  test_size=0.4, random_state=42)
X_val, X_test= train_test_split(X_test,  test_size=0.5, random_state=42)
X_train.head()

In [ ]:
import pandas as pd
seq = X_train['new_utterance'].tolist()
pd.DataFrame(seq).to_csv("./train_dev", index=False, header=None)

In [ ]:
import pandas as pd
seq = X_test['new_utterance'].tolist()
pd.DataFrame(seq).to_csv("./test", index=False, header=None)

# construct input for context model

In [ ]:
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv("45k_add_slot_label.csv")

In [ ]:
context_data = []
conversation_id_old = -1
for index, row in data.iterrows():
  conversation_id = row['conversation_id']
  if conversation_id!=conversation_id_old:
    context_data.append(row['word_level_data'])
  else:
    old_context = context_data[-1]
    context_data.append(str(old_context)+' [eos] '+str(row['word_level_data']))
  conversation_id_old = conversation_id

In [ ]:
context_data_depth3 = []
souce_back_depth = 7
conversation_id_old = -1
for context in context_data:
  context = str(context)
  context_split = context.split('[eos]')
  if len(context_split)<=souce_back_depth:
    context_data_depth3.append(context)
  else:
    context_split_reserve = context_split[-souce_back_depth:]
    context_data_depth3.append('[eos]'.join(context_split_reserve))
print(len(context_data_depth3))

In [ ]:
data['context_data']=context_data

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data,  test_size=0.4, random_state=42)
X_val, X_test= train_test_split(X_test,  test_size=0.5, random_state=42)

In [ ]:
import re

train_seqin=[]

train_label=[]
train_seqout = []

train_seqin = X_train['context_data'].tolist()
train_label= X_train['final_label'].tolist()
train_seqout = X_train['slot_label'].tolist()
print(len(train_seqout))
print(len(train_seqin))
print(len(train_label))

pd.DataFrame(train_seqin).to_csv("./seq.in", index=False, header=None)

pd.DataFrame(train_seqout).to_csv("./seq.out", index=False, header=None)
pd.DataFrame(train_label).to_csv("./label", index=False, header=None)

In [ ]:
import re

train_seqin=[]

train_label=[]
train_seqout = []

train_seqin = X_val['context_data'].tolist()
train_label= X_val['final_label'].tolist()
train_seqout = X_val['slot_label'].tolist()
print(len(train_seqout))
print(len(train_seqin))
print(len(train_label))

pd.DataFrame(train_seqin).to_csv("./seq.in", index=False, header=None)

pd.DataFrame(train_seqout).to_csv("./seq.out", index=False, header=None)
pd.DataFrame(train_label).to_csv("./label", index=False, header=None)

In [ ]:
import re

train_seqin=[]

train_label=[]
train_seqout = []

train_seqin = X_test['word_level_data'].tolist()
train_label= X_test['final_label'].tolist()
train_seqout = X_test['slot_label'].tolist()
print(len(train_seqout))
print(len(train_seqin))
print(len(train_label))

pd.DataFrame(train_seqin).to_csv("./seq.in", index=False, header=None)

pd.DataFrame(train_seqout).to_csv("./seq.out", index=False, header=None)
pd.DataFrame(train_label).to_csv("./label", index=False, header=None)